In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
domainSz = np.array([150, 50, 50]) # in mm

In [4]:
cellSz = np.array([10, 10, 10]) # in mm
numCells = (domainSz//cellSz).astype(int) # TODO: ensure domainSz is divisible by cellSz

voxelSz = np.array([1, 1, 1]) # in mm
#voxelSz = np.array([0.5, 0.5, 0.5]) # in mm
numVoxels = (cellSz//voxelSz).astype(int) # in each cell/unit

In [5]:
paramsDim = 1 # number of parameters for each cell
params = 2.5*np.ones(np.append(numCells, paramsDim))

In [6]:
# origin coordinates of cells
x_c, y_c, z_c = np.meshgrid(range(numCells[0])*cellSz[0], 
                            range(numCells[1])*cellSz[1], 
                            range(numCells[2])*cellSz[2], indexing='ij')

# voxel center coordinates relative to cell origin
x_v, y_v, z_v = np.meshgrid(range(numVoxels[0])*voxelSz[0] + voxelSz[0]/2, 
                            range(numVoxels[1])*voxelSz[1] + voxelSz[1]/2, 
                            range(numVoxels[2])*voxelSz[2] + voxelSz[2]/2, indexing='ij')

In [7]:
field = np.zeros(numCells*numVoxels)
holes = np.zeros((np.prod(numCells), 3)) # hole coordinates in the cells, only for this type of cells
hh = 0
# TODO: make more efficient by doing vectorized computations outside loop
for i in range(numCells[0]):
    for j in range(numCells[1]):
        for k in range(numCells[2]):
            p = params[i, j, k]
            # voxel center global coordinates
            x = x_v + x_c[i, j, k]
            y = y_v + y_c[i, j, k]
            z = z_v + z_c[i, j, k]
            # cell center coordinates
            xx = x_c[i, j, k] + cellSz[0]/2
            yy = y_c[i, j, k] + cellSz[1]/2
            zz = z_c[i, j, k] + cellSz[2]/2
            holes[hh] = [xx, yy, zz]
            hh += 1
            # calculate distance field
            cell = np.sqrt((x - xx)**2 + (y - yy)**2 + (z - zz)**2) - p
            field[i*numVoxels[0]:(i+1)*numVoxels[0], 
                  j*numVoxels[1]:(j+1)*numVoxels[1], 
                  k*numVoxels[2]:(k+1)*numVoxels[2]] += cell

In [9]:
from lattice import field_to_inp

field_to_inp(field, "data/lattice.inp", holes, 0., voxelSz)

Field converted to Surf!
Surf converted to INP
Field converted to INP


In [ ]:
# ALL FOLLOWING CELLS ONLY FOR TEXTING


field = np.pad(field, 1, 'constant', constant_values = 0.)

#plt.matshow(field[8, :, :])
#plt.colorbar()
#plt.show()

In [ ]:
surf_mesh = trimesh.Trimesh(verts, faces)

surf_mesh.export('data/lattice.stl', 'stl_ascii')
#surf_mesh.show()

In [ ]:
# Jun Test Case
surf_mesh = trimesh.load('data/SVLP_f1_R40.stl')

verts = surf_mesh.vertices
faces = surf_mesh.faces

faces = np.fliplr(faces)

holes = []

In [ ]:
from meshpy.tet import MeshInfo, build
from meshpy.geometry import GeometryBuilder

In [ ]:
builder = GeometryBuilder()
builder.add_geometry(
    points=verts.tolist(),
    facets=faces.tolist())
builder.wrap_in_box(1)

mi = MeshInfo()
builder.set(mi)
mi.set_holes(holes)
mesh = build(mi)
print("%d elements" % len(mesh.elements))
#mesh.write_vtk("out.vtk")

In [ ]:
def header(outFile, commentLine, keywordLine):
    """Writes header on file object

    outFile, file object: already opened file object to write
    commentLine, str: string which goes in as comment
    keywordLine, str: keyword line with parameters (if required)
    Examples
    >>> header(testfile, "Nodal Coordinates", "NODE")
    >>> header(testfile, "Elements", "ELEMENT, TYPE=S4")
    """
    outFile.write("**\n")
    outFile.write("%s\n" %("*"*(len(commentLine) + 3)))
    outFile.write("** %s\n" %(commentLine))
    outFile.write("%s\n" %("*"*(len(commentLine) + 3)))
    outFile.write("*%s\n" %(keywordLine))

In [ ]:
def writeNodeLine(outFile, nodeNumber, coords):
    """Writes the nodal coordinates line in Abq format, doesnt return anything

    outFile, file object: already opened file object to write
    nodeNumber, int: node number
    coords, list of float: list of coordinates of current node(<x1>,<x2>,<x3>)
    """
    outFile.write("%d, %g, %g, %g\n" %(nodeNumber, coords[0],
                                                      coords[1], coords[2]))
# Similarly for Element definition
def writeElemLine(outFile, elemNumber, nodalCnvty):
    """Writes the Element line in Abq format, doesnt return anything; This is
    valid only for 4 noded shell elements.

    outFile, file object: already opened file object to write
    elemNumber, int: Element number
    nodalCnvty, list of int: Nodal connectivity of current element (<node1>,
                            <node2>, <node3>, <node4>)
    """
    outFile.write("%d, %d, %d, %d, %d\n" %(elemNumber, \
                nodalCnvty[0], nodalCnvty[1], nodalCnvty[2], nodalCnvty[3]))

In [ ]:
nodes = np.asarray(mesh.points)
elems = np.asarray(mesh.elements) + 1

with open("data/lattice.inp", "w") as inp:
    header(inp, "Nodal Coordinates", "NODE")    # Write node header
    # write nodes
    for i  in range(nodes.shape[0]):
        writeNodeLine(inp, i+1, nodes[i])

    inp.write("\n")
    header(inp, "Elements", "ELEMENT, TYPE=C3D4")    # Write Element header
    for i  in range(elems.shape[0]):
        writeElemLine(inp, i+1, elems[i])